# openpmd-scipp
Load openpmd datasets to `scipp` `DataArrays`.

## Description
### What is this good for?
[`scipp`](https://github.com/scipp/scipp) is an alternative to [`xarray`](https://github.com/pydata/xarray) and provides basically numpy arrays with axes description and units.
* Automatically load axes and units with openPMD data.
* Axes information  is automatically updated when slicing, indexing, or filtering your data.
* With `scipp`'s plotting library [`plopp`](https://github.com/scipp/plopp) it becomes an alternative to `openpmd-viewer`.
* Many numpy and some scipy functions including all the basic algebraic operations on arrays are supported by `scipp`. When using these, the units and coordinates are automatically taken care of. 

### Limitations
* `scipp` currently handles units with a library, that does not support non-integer exponents for units. This can become problematic in some calculations. 



## Installation
It can be easily installed with pip.
```bash
git clone https://github.com/pordyna/openpmd_scipp.git
cd openpmd-scipp
pip install .
```

## Getting started
Get example data sets from the `openPMD-example-datasets` repository.
```bash
git clone https://github.com/openPMD/openPMD-example-datasets.git
cd openPMD-example-datasets
tar -zxvf example-2d.tar.gz
tar -zxvf example-3d.tar.gz
```
 

### Opening series

In [ ]:
import openpmd_scipp as pmdsc
import scipp as sc

In [ ]:
path = "openPMD-example-datasets/example-3d/hdf5/data%T.h5"

In [ ]:
path = ".data/" + path

In [ ]:
data_loader = pmdsc.DataLoader(path)
print(data_loader.iterations)

### Working with meshes (fields)
Let us plot electric field's x component at 65 fs. 


In [ ]:
Ex = data_loader.get_field("E", "x", time=65 * sc.Unit("fs"))
print(Ex)

You may have noticed, that the time requested does not have to match exactly any iteration. By default, if there is an iteration within 10 fs distance it will be used instead. This 10 fs tolerance can be adjusted by setting `time_tolerance`. The check can be also disabled by setting `time_tolerance=None`, with that the method will return the closest iteration regardless of the difference. So that this will also work:

In [ ]:
print(data_loader.get_field("E", "x", time=20 * sc.Unit("fs"), time_tolerance=20 * sc.Unit("fs")))

, but `data_loader.get_field('E', 'x', time=20 * sc.Unit('fs'))` not.

In [ ]:
# It is also possible to use iteration number instead:
print(data_loader.get_field("E", "x", iteration=200))

In [ ]:
# For scalar fields just omit the second argument:
print(data_loader.get_field("rho", iteration=200))

#### Plotting
W can't directly plot 3D data.
But we can for example select a slice. For that we can use a helper function `pmdsc.closest` to get the closets index, since `scipp` requires exact match. You can read more about indexing `scipp` arrays in `scipp`'s documentation.

In [ ]:
slicing_idx = pmdsc.closest(Ex, "x", 2 * sc.Unit("um"))
Ex_slice = Ex["x", slicing_idx]
print(Ex_slice)

In [ ]:
Ex_slice.plot()

In [ ]:
# We can also plot line plots:
Ex_line = Ex_slice["z", pmdsc.closest(Ex_slice, "z", 1.4e-5 * sc.Unit("m"))]
print(Ex_line)

In [ ]:
Ex_line.plot()

Alternatively it is possible to work interactively with `plopp`'s tools for visualizing multidimensional data, such as `pp.slicer`or `pp.inspector`.

#### Doing math
Just as an example we can easily plot the square of the field:

In [ ]:
(Ex_line * Ex_line).plot()

### Loading chunks
In the above example the whole 3D field is loaded into memory and sliced afterward. It is also possible to just load a sub-chunk into memory. When the `relay` option in `get_field` is set to `True` it will return a dummy object that only allocates memory for a single value. This relay object can be indexed, sliced etc. using the `scipp` indexing just like before. (The only limitation given by the `openpmd-api` is that the result has to a be contiguous chunk of the original array). The `load_data` method loads data and returns a proper `scipp` data array. 

In [ ]:
# The full 3D array is not loaded into memory at this point.
Ex = data_loader.get_field("E", "x", time=65 * sc.Unit("fs"), relay=True)
# This time we will select a range rather than a slice.
# For a range there is no need for an exact match.
# But, we could also select a slice just like in the previous example.
Ex = Ex["x", -2e-6 * sc.Unit("m") : 2e-6 * sc.Unit("m")]
# Only now the smaller subset wil be loaded into memory
Ex = Ex.load_data()
print(Ex)

### Time axis
It is also possible to combine arrays from different iterations into one using `scipp`'s `concat` function. Here is an example for creating a 4D array from all iterations:

In [ ]:
Ex = sc.concat(
    [
        data_loader.get_field("E", "x", iteration=iteration.value, time_tolerance=None)
        for iteration in data_loader.iterations["iteration_id"]
    ],
    dim="t",
)
print(Ex)

The reason for the z coordinate having two dimensions (t,z) is the fact that the data comes from a moving window simulation. This is clearly visible in the plot below.

In [ ]:
# Let us just slice at some points to get a 2D dataset
Ex = Ex["x", 10]["y", 10]
print(Ex)

In [ ]:
Ex.plot()

### Working with particle data
Coming soon!

## Developer documentation
### Generating this README
README file is generated from the README.ipynb.
```
# Download and extract example datasets if not present
# Will download data into `.data`
make data

make docs
```
### Running tests
At the moment we only test we have is an integration test running this notebook. After downloading example datasets with `make data`, if needed, run:
```
make test
```
You can also run tests with different python version with tox, but you need to have the python version installed, for example with pyenv.
